In [1]:
import os
import glob
import numpy as np
import xarray as xr
import time
import itertools

import sys
p = os.path.abspath('/global/homes/q/qnicolas/')
if p not in sys.path:
    sys.path.append(p)

from tools.e5tools import *
xr.set_options(display_style='text') 

In [2]:
mountains=         {"vietnam"    :([102,115,10,19.5],"Annamite range (Vietnam)",240,10),
                    "ghats"      :([68,80,7.5,17]   ,"Western Ghats"           ,70 ,6 ),
                    "madagascar" :([40,60,-25,-12]  ,"Madagascar"              ,280,3 ),
                    "myanmar"    :([80,102,7,27]    ,"Myanmar"                 ,60 ,7 ),
                    "newbritain" :([148,155,-12,0]  ,"New Britain"             ,320,7 ),
                    "philippines":([118,130,10,22]  ,"Philippines"             ,225,12),
                    "sumatra"    :([90,110,-6,6]    ,"Bukit Barisan (Sumatra)" ,70 ,11),
                    "malaysia"   :([97,107,1,10]    ,"Malaysia"                ,225,11),
                    "bayofbengal":([82,90,12,18],),
                    "ghatswide"  :([68,85,7.5,17]  ,),
                    "myanmarwide"     :([85,105,12,29]  ,),
                    "newbritainwide"  :([146,155,-12,0]  ,),
                   }

precip_boxes = {"vietnam"    :[107  ,  19  , 110.5, 14   ,2  ],
                "ghats"      :[ 75  ,   9  ,  72.5, 16   ,2  ],
                "madagascar" :[ 52  , -14.5,  49  , -24.5,2.5],
                "myanmar"    :[ 98  ,  11  ,  90  , 21   ,4  ],
                "newbritain" :[154  , - 6  , 150  , -8   ,2  ],
                "philippines":[123.5,  19  , 127  , 11   ,3  ],
                "sumatra"    :[ 98.5, - 2  ,  96  , 2    ,2.5],
                "malaysia"   :[100.5,  10  , 105  , 4    ,2  ],
               }

In [3]:
def tilted_rect(grid,x1,y1,x2,y2,width,reverse=False):
    x = grid.longitude
    y = grid.latitude
    if reverse:
        halfplane_para = (x-x1)*(y2-y1) - (x2-x1)*(y-y1) <=0
    else:
        halfplane_para = (x-x1)*(y2-y1) - (x2-x1)*(y-y1) >=0
    sc_prod = (x-x1)*(x2-x1)+(y-y1)*(y2-y1)
    halfplane_perp_up = sc_prod >= 0
    halfplane_perp_dn = (x-x2)*(x1-x2)+(y-y2)*(y1-y2) >= 0
    distance_across = np.sqrt((x-x1)**2+(y-y1)**2 - sc_prod**2/((x2-x1)**2+(y2-y1)**2))
    return (halfplane_para*halfplane_perp_up*halfplane_perp_dn*(distance_across<width)).transpose('latitude','longitude')
def crossslopeflow(u,v,angle):
    return (u*np.sin(angle*np.pi/180)+v*np.cos(angle*np.pi/180))
    

# CLIMCAPS sdev - temporary

In [1]:
paths=[]
with open("/global/cscratch1/sd/qnicolas/CLIMCAPS_V2/url_daily.txt",'r') as f:
    lines = f.readlines()
    for f in lines:
        if f[-58:-52] == "201310":
            paths.append(f[-81:-1])

In [ ]:
import xarray as xr
from pydap.client import open_url
from pydap.cas.urs import setup_session
import pandas as pd
import time

username = 'qnicolas'
password = 'Surf@b3rkeley!'

urls = ["https://sounder.gesdisc.eosdis.nasa.gov/opendap/Aqua_Sounder_Level3/SNDRAQIML3CDCCP.2/"+ path for path in paths]
url_ex = urls[0]

session = setup_session(username, password, check_url=url_ex)
store = xr.backends.PydapDataStore.open(url_ex, session=session)
ds = xr.open_dataset(store)

climcaps_sdevs_201310 = 0.+0.*ds.sdev_air_temp_sdev.expand_dims({'time':pd.date_range("20131001","20131010")})

In [6]:
for i,url in enumerate(urls[:10]):
    t=time.time()
    session = setup_session(username, password, check_url=url)
    store = xr.backends.PydapDataStore.open(url, session=session)
    ds = xr.open_dataset(store)
    climcaps_sdevs_201310[i]=ds.sdev_air_temp_sdev
    print(time.time()-t)
    print("")

44.68144130706787

62.95948815345764

72.17381429672241

74.03774404525757

78.38177537918091

36.24507117271423

83.1578996181488

85.92022728919983

61.21811366081238

83.58768391609192



# Computing daily&box means

In [4]:
years=range(1998,2014)
months_perregion = {"vietnam"    :[10],
                    "ghats"      :[7],
                    "madagascar" :[3,4],
                    "myanmar"    :[6,7,8],
                    "newbritain" :[6,7,8],
                    "philippines":[11,12],
                    "sumatra"    :[1,11,12],
                    "malaysia"   :[11,12],
                    "myanmarwide"   :[6,7,8],
                    "newbritainwide":[6,7,8],
                   }
#"ghats"      :[6,7,8],#"vietnam"      :[10,11],

#months_perregion = {"vietnam"    :[10],
#                    "ghats"      :[7],
#                    "madagascar" :[3],
#                    "myanmar"    :[7],
#                    "newbritain" :[7],
#                    "philippines":[12],
#                    "sumatra"    :[11],
#                    "malaysia"   :[11],
#                    "bayofbengal":[7],
#                    "ghatswide":[6,8],
#                   }

## 1 - Create zarr groups

In [5]:
from dask.distributed import Client
client = Client()
client

/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33163 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:35161 Dashboard: http://127.0.0.1:33163/status,Cluster Workers: 8 Cores: 64 Memory: 540.14 GB


In [7]:
remaining_regions = "madagascar","myanmar","newbritain","philippines","sumatra","malaysia"

In [12]:
years=range(2002,2014)
CLIMCAPS_PATH = "/global/cscratch1/sd/qnicolas/CLIMCAPS_V2/"
for name in ['ghats']:
    print(name)
    box=mountains[name][0]
    months=months_perregion[name]
    for m in months:
        print(m)
        preprocess = lambda ds: ds.swap_dims({'orbit_pass':'obs_time_tai93'}).rename(obs_time_tai93='time',lat='latitude',lon='longitude').spec_hum.isel(time=[1,0])
        ds = xr.open_mfdataset(sorted(glob.glob(CLIMCAPS_PATH+"daily/SNDR.AQUA.AIRS_IM.????{:02}*.L3_CLIMCAPS_QCC.std.v02_38.G.*.nc".format(m))),preprocess=preprocess,combine='nested',concat_dim='time',parallel=True)
        %time ds.reindex(latitude=list(reversed(ds.latitude))).sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_dataset().to_zarr("/global/cscratch1/sd/qnicolas/regionsData/L3_CLIMCAPS_QCC.spec_hum.2002-2015.{:02}.{}.zarr".format(m,name),mode="w")
        
        
        

ghats
7


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 9.04 s, sys: 888 ms, total: 9.93 s
Wall time: 36.8 s


In [99]:
years=range(2002,2014)
CLIMCAPS_PATH = "/global/cscratch1/sd/qnicolas/CLIMCAPS_V2/"

preprocess = lambda ds: ds.air_temp.assign_coords({'time':np.array(ds.obs_time_tai93[0])}).expand_dims('time').rename(lat='latitude',lon='longitude')
ds = xr.open_mfdataset(sorted(glob.glob(CLIMCAPS_PATH+"monthly/SNDR.AQUA.AIRS_IM.*.nc")),preprocess=preprocess,combine='nested',concat_dim='time',parallel=True)
%time ds.reindex(latitude=list(reversed(ds.latitude))).to_dataset().to_zarr(CLIMCAPS_PATH+"monthly/L3_CLIMCAPS_QCC.air_temp.monthly.zarr",mode="w")
        
        
        

/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 3.37 s, sys: 301 ms, total: 3.67 s
Wall time: 14.8 s


In [ ]:
print(1)

In [6]:
years=range(2002,2016)
CLIMCAPS_PATH = "/global/cscratch1/sd/qnicolas/CLIMCAPS_V2/"
for name in ['myanmarwide','newbritainwide']:
    print(name)
    box=mountains[name][0]
    months=months_perregion[name]
    for m in months:
        print(m)
        preprocess = lambda ds: ds.swap_dims({'orbit_pass':'obs_time_tai93'}).rename(obs_time_tai93='time',lat='latitude',lon='longitude').air_temp.isel(time=[1,0])
        ds = xr.open_mfdataset(sorted(glob.glob(CLIMCAPS_PATH+"daily/SNDR.AQUA.AIRS_IM.????{:02}*.L3_CLIMCAPS_QCC.std.v02_38.G.*.nc".format(m))),preprocess=preprocess,combine='nested',concat_dim='time',parallel=True)
        %time ds.reindex(latitude=list(reversed(ds.latitude))).sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_dataset().to_zarr("/global/cscratch1/sd/qnicolas/regionsData/L3_CLIMCAPS_QCC.air_temp.2002-2015.{:02}.{}.zarr".format(m,name),mode="w")
        
        
        

myanmarwide
6


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 7.65 s, sys: 988 ms, total: 8.64 s
Wall time: 34.4 s
7


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 7.1 s, sys: 914 ms, total: 8.01 s
Wall time: 34.6 s
8


...

CPU times: user 7.11 s, sys: 833 ms, total: 7.94 s
Wall time: 37.7 s
7


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 8.72 s, sys: 859 ms, total: 9.57 s
Wall time: 43.9 s
8


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 8.11 s, sys: 902 ms, total: 9.01 s
Wall time: 41.8 s


In [6]:
years=range(2001,2016)
for varcode in ["128_132_v"]:
    for name in ['vietnam','myanmarwide','newbritainwide']:
        print(varcode,name)
        box=mountains[name][0]
        months=months_perregion[name]
        for y,m in itertools.product(years,months):
            print("{}{:02}".format(y,m))
            ds = xr.open_mfdataset(ERA5D_PATH+"e5.oper.an.pl/{}{:02}/e5.oper.an.pl.{}.*.nc".format(y,m,varcode))
            %time ds.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name),mode="w")


128_132_v vietnam
200110
CPU times: user 11.5 s, sys: 957 ms, total: 12.4 s
Wall time: 2min 14s
200210
CPU times: user 10.6 s, sys: 871 ms, total: 11.5 s
Wall time: 2min 4s
200310
CPU times: user 10.2 s, sys: 997 ms, total: 11.2 s
Wall time: 2min 3s
200410
CPU times: user 10.3 s, sys: 950 ms, total: 11.3 s
Wall time: 2min 4s
200510
CPU times: user 10.3 s, sys: 950 ms, total: 11.2 s
Wall time: 2min 5s
200610
CPU times: user 10.3 s, sys: 932 ms, total: 11.2 s
Wall time: 2min 1s
200810
CPU times: user 9.68 s, sys: 1.22 s, total: 10.9 s
Wall time: 2min 1s
200910
CPU times: user 9.64 s, sys: 1.16 s, total: 10.8 s
Wall time: 2min 3s
201010
CPU times: user 9.58 s, sys: 1.22 s, total: 10.8 s
Wall time: 2min 1s
201110
CPU times: user 9.67 s, sys: 1.33 s, total: 11 s
Wall time: 2min 5s
201210
CPU times: user 10.2 s, sys: 1.41 s, total: 11.6 s
Wall time: 2min 10s
201310
CPU times: user 10.3 s, sys: 1.41 s, total: 11.7 s
Wall time: 2min 10s
201410
CPU times: user 10 s, sys: 1.3 s, total: 11.3 s
Wa

In [6]:
def createzarr_era5(name,varcode):
    box=mountains[name][0]
    years=range(2001,2016)
    months=months_perregion[name]
    for m in months:
        var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.sfc/*/e5.oper.an.sfc.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
        var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name),mode="w")
        

In [7]:
def createzarr_trmm(name):
    box=mountains[name][0]
    years=range(2001,2014)
    months=months_perregion[name]
    for m in months:
        var = xr.open_mfdataset([glob.glob(CMIP6_FOLDER+"obs4mip/NASA-GSFC/TRMM/observations/atmos/pr/3hr/NASA-GSFC/TRMM/*/*_{}{:02}*.nc".format(y,m))[0] for y in years])
        var = var.rename({'lat':'latitude','lon':'longitude'})
        var = var.reindex(latitude=list(reversed(var.latitude)))
        var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr("/global/cscratch1/sd/qnicolas/regionsData/trmm.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name),mode="w")


In [8]:
def flatten(t):
    return [item for sublist in t for item in sublist]
def createzarr_gpm(name):
    box=mountains[name][0]
    years=range(2001,2016)
    months=months_perregion[name]
    for m in months:
        var = xr.open_mfdataset([glob.glob("/global/cscratch1/sd/pnicknis/gpm/daily/3B-DAY.MS.MRG.3IMERG.{}{:02}*-S000000-E235959.V06.nc4".format(y,m)) for y in years])
        var = var.rename({'lat':'latitude','lon':'longitude'})
        var = var.reindex(latitude=list(reversed(var.latitude)))
        var['time'] = var.indexes['time'].to_datetimeindex()
        var.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2])).to_zarr("/global/cscratch1/sd/qnicolas/regionsData/gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name),mode="w")


In [9]:
%%time 
for name in ['myanmarwide','newbritainwide']:
    createzarr_gpm(name)
    createzarr_era5(name,"228_246_100u")

/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return se

CPU times: user 7min 26s, sys: 2min 16s, total: 9min 42s
Wall time: 15min 36s


## Time and space mean

### Daily

In [11]:
def resample_daily(ds):
    """performs the same job as resample(time="1D") but keeps missing days out"""
    ds.coords['time'] = ds.time.dt.floor('1D')
    return ds.groupby('time').mean()

In [37]:
# 40secs
%%time 
q_spatialmeans=[]
for y,m in itertools.product(years,months):
    qzarr = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.{}{:02}.ghats.zarr".format(y,m)).Q
    mask_ghats_era5= tilted_rect(qzarr,*precip_boxes["ghats"],reverse=True)
    q_spatialmeans.append(spatial_mean(resample_daily(qzarr),box=None,mask=mask_ghats_era5))

temp=xr.concat(q_spatialmeans,dim='time')
temp.to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.1998-2013.JJA.ghats.upstreammean.nc")    


In [9]:
%%time 
q_spatialmeans=[]
for y,m in itertools.product(years,months):
    qzarr = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.{}{:02}.ghats.zarr".format(y,m)).Q
    mask_ghats_era5_above= tilted_rect(qzarr,*precip_boxes["ghats"])
    q_spatialmeans.append(spatial_mean(resample_daily(qzarr),box=None,mask=mask_ghats_era5_above))

temp=xr.concat(q_spatialmeans,dim='time')
temp.to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.1998-2013.JJA.ghats.mean.nc")    


CPU times: user 46.1 s, sys: 16.5 s, total: 1min 2s
Wall time: 45.4 s


In [7]:
# Do June and July-august separately because they have a different number of days, so chunk size differ and saving to zarr would require rechunking
def timespacemean_era5_JJA(name,varcode):
    var_J =xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/e5.oper.an.sfc.%s.ll025sc.1998-2013.J.%s.zarr"%(varcode,name))
    var_JA=xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/e5.oper.an.sfc.%s.ll025sc.1998-2013.JA.%s.zarr"%(varcode,name))
    varname = list(var_J.data_vars)[0]
    mask_ghats_era5= tilted_rect(var_J,*precip_boxes[name],reverse=True)
    var_J_sm  = spatial_mean(resample_daily( var_J[varname]),box=None,mask=mask_ghats_era5)
    var_JA_sm = spatial_mean(resample_daily(var_JA[varname]),box=None,mask=mask_ghats_era5)
    xr.concat([var_J_sm,var_JA_sm],dim='time').sortby('time').to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.sfc.%s.ll025sc.1998-2013.JJA.%s.upstreammean.nc"%(varcode,name))
    
def timespacemean_trmm_JJA(name):
    var_J =xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/trmm.1998-2013.J.%s.zarr"%name)
    var_JA=xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/trmm.1998-2013.JA.%s.zarr"%name)
    mask_ghats_trmm= tilted_rect(var_J,*precip_boxes[name])
    var_J_sm  = 86400*spatial_mean(resample_daily( var_J.pr),box=None,mask=mask_ghats_trmm)
    var_JA_sm = 86400*spatial_mean(resample_daily(var_JA.pr),box=None,mask=mask_ghats_trmm)
    xr.concat([var_J_sm,var_JA_sm],dim='time').sortby('time').to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/trmm.1998-2013.JJA.%s.mean.nc"%name)    
    

In [9]:
for name in ["ghats,""myanmar"    ,"newbritain"]:
    timespacemean_era5_JJA(name,"228_246_100u")
    timespacemean_era5_JJA(name,"228_247_100v")
    timespacemean_trmm_JJA(name)

In [7]:
%%time 
monthsinitials=['J','F','M','A','M','J','J','A','S','O','N','D']
for name in ["vietnam","madagascar","philippines","sumatra","malaysia"]:
    print(name)
    months=months_perregion[name]
    q_spatialmeans=[]
    for y,m in itertools.product(years,months):
        qzarr = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.{}{:02}.{}.zarr".format(y,m,name)).Q
        mask_era5= tilted_rect(qzarr,*precip_boxes[name],reverse=True)
        q_spatialmeans.append(spatial_mean(resample_daily(qzarr),box=None,mask=mask_era5))
    temp=xr.concat(q_spatialmeans,dim='time')
    temp.to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.1998-2013.%s.%s.upstreammean.nc"%(''.join([monthsinitials[m-1] for m in months]),name))


vietnam
madagascar
philippines
sumatra
malaysia
CPU times: user 2min 13s, sys: 8.38 s, total: 2min 21s
Wall time: 3min 19s


In [12]:
%%time 
monthsinitials=['J','F','M','A','M','J','J','A','S','O','N','D']
for name in mountains:
    print(name)
    months=months_perregion[name]
    t_spatialmeans=[]
    for y,m in itertools.product(years,months):
        tzarr = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_130_t.ll025sc.{}{:02}.{}.zarr".format(y,m,name)).T
        mask_era5= tilted_rect(tzarr,*precip_boxes[name],reverse=True)
        t_spatialmeans.append(spatial_mean(resample_daily(tzarr),box=None,mask=mask_era5))
    temp=xr.concat(t_spatialmeans,dim='time')
    temp.to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_130_t.ll025sc.1998-2013.%s.%s.upstreammean.nc"%(''.join([monthsinitials[m-1] for m in months]),name))


vietnam
ghats
madagascar
myanmar
newbritain
philippines
sumatra
malaysia
CPU times: user 3min 55s, sys: 13 s, total: 4min 8s
Wall time: 5min 57s


In [17]:
def timespacemean_era5(name,varcode):
    months=months_perregion[name]
    var_months = [xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/e5.oper.an.sfc.{}.ll025sc.1998-2013.{:02}.{}.zarr".format(varcode,m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    mask_ghats_era5= tilted_rect(var_months[0],*precip_boxes[name],reverse=True)
    var_months_sm  = [spatial_mean(resample_daily(var[varname]),box=None,mask=mask_ghats_era5) for var in var_months]
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.sfc.%s.ll025sc.1998-2013.%s.%s.upstreammean.nc"%(varcode,''.join([monthsinitials[m-1] for m in months]),name))
    
def timespacemean_trmm(name):
    months=months_perregion[name]
    var_months = [xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/trmm.1998-2013.{:02}.{}.zarr".format(m,name)) for m in months]
    mask_ghats_trmm= tilted_rect(var_months[0],*precip_boxes[name])
    var_months_sm  = [86400*spatial_mean(resample_daily( var.pr),box=None,mask=mask_ghats_trmm) for var in var_months]
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/trmm.1998-2013.%s.%s.mean.nc"%(''.join([monthsinitials[m-1] for m in months]),name))    
    

In [17]:
for name in ["vietnam","madagascar","philippines","sumatra","malaysia"]:
    %time timespacemean_era5(name,"228_246_100u")
    %time timespacemean_era5(name,"228_247_100v")
    %time timespacemean_trmm(name)

CPU times: user 19.8 s, sys: 914 ms, total: 20.7 s
Wall time: 22.9 s
CPU times: user 19.5 s, sys: 805 ms, total: 20.3 s
Wall time: 22.8 s
CPU times: user 21.8 s, sys: 967 ms, total: 22.7 s
Wall time: 24.9 s
CPU times: user 19.9 s, sys: 774 ms, total: 20.7 s
Wall time: 22.7 s
CPU times: user 19.3 s, sys: 703 ms, total: 20 s
Wall time: 22.2 s
CPU times: user 18.7 s, sys: 636 ms, total: 19.3 s
Wall time: 21.6 s
CPU times: user 18.4 s, sys: 802 ms, total: 19.2 s
Wall time: 22 s
CPU times: user 18.5 s, sys: 861 ms, total: 19.4 s
Wall time: 21.5 s
CPU times: user 21.1 s, sys: 1.05 s, total: 22.2 s
Wall time: 24.4 s
CPU times: user 29.2 s, sys: 893 ms, total: 30.1 s
Wall time: 32.9 s
CPU times: user 28.9 s, sys: 1.16 s, total: 30.1 s
Wall time: 33 s
CPU times: user 30.4 s, sys: 962 ms, total: 31.4 s
Wall time: 34.8 s
CPU times: user 19 s, sys: 589 ms, total: 19.6 s
Wall time: 22.1 s
CPU times: user 20.1 s, sys: 922 ms, total: 21 s
Wall time: 23.4 s
CPU times: user 19.2 s, sys: 591 ms, total: 

### 12-hourly

In [27]:
%%time 
q_spatialmeans=[]
for y,m in itertools.product(years,months):
    qzarr = xr.open_zarr("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.{}{:02}.ghats.zarr".format(y,m)).Q
    mask_ghats_era5_above= tilted_rect(qzarr,*precip_boxes["ghats"])
    q_spatialmeans.append(spatial_mean(qzarr.coarsen(time=12).mean(),box=None,mask=mask_ghats_era5_above))

temp=xr.concat(q_spatialmeans,dim='time')
temp['time']=temp['time']+pd.Timedelta("30m")
temp.to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.pl.128_133_q.ll025sc.1998-2013.JJA.12h.ghats.mean.nc")    


CPU times: user 39.9 s, sys: 14.1 s, total: 54 s
Wall time: 14.8 s


In [28]:
# Do June and July-august separately because they have a different number of days, so chunk size differ and saving to zarr would require rechunking
def halfdayspacemean_era5(varcode):
    var_J =xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/e5.oper.an.sfc.%s.ll025sc.1998-2013.J.ghats.zarr"%varcode)
    var_JA=xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/e5.oper.an.sfc.%s.ll025sc.1998-2013.JA.ghats.zarr"%varcode)
    varname = list(var_J.data_vars)[0]
    mask_ghats_era5= tilted_rect(var_J,*precip_boxes["ghats"],reverse=True)
    var_J_sm  = spatial_mean( var_J[varname].coarsen(time=12).mean(),box=None,mask=mask_ghats_era5)
    var_JA_sm = spatial_mean(var_JA[varname].coarsen(time=12).mean(),box=None,mask=mask_ghats_era5)
    temp = xr.concat([var_J_sm,var_JA_sm],dim='time').sortby('time')
    temp['time']=temp['time']+pd.Timedelta("30m")
    temp.to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/e5.oper.an.sfc.%s.ll025sc.1998-2013.JJA.12h.ghats.upstreammean.nc"%varcode)
    
def halfdayspacemean_trmm():
    var_J =xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/trmm.1998-2013.J.ghats.zarr")
    var_JA=xr.open_zarr("/global/cscratch1/sd/qnicolas/temp/trmm.1998-2013.JA.ghats.zarr")
    mask_ghats_trmm= tilted_rect(var_J,*precip_boxes["ghats"])
    var_J_sm  = 86400*spatial_mean( var_J.pr.coarsen(time=4).mean(),box=None,mask=mask_ghats_trmm)
    var_JA_sm = 86400*spatial_mean(var_JA.pr.coarsen(time=4).mean(),box=None,mask=mask_ghats_trmm)
    xr.concat([var_J_sm,var_JA_sm],dim='time').sortby('time').to_netcdf("/global/cscratch1/sd/qnicolas/regionsData/trmm.1998-2013.JJA.12h.ghats.mean.nc")    
    

In [29]:
%%time 
halfdayspacemean_era5("228_246_100u")
halfdayspacemean_era5("228_247_100v")
halfdayspacemean_trmm()

CPU times: user 3.42 s, sys: 955 ms, total: 4.38 s
Wall time: 1.19 s
